* I experiment with getting the "anomaly score" using Isolation forests, and add that as a feature and see if it improves our supervised fraud prediction task. 
    * It does!
* Isolation forest and other semisupervised approaches:
    * https://www.kaggle.io/svf/1100683/56c8356ed1b0a6efccea8371bc791ba7/__results__.html#Tree-based-techniques

* Kernel uses the imbalanced-learn (imblearn) library for boilerplate under (or over) sampling.
    * pip install imblearn
### V2:
    * This version of the notebook skips the downsampling
    * ORIGINAL: https://www.kaggle.com/danofer/anomaly-detection-for-feature-engineering

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, IsolationForest

from sklearn.model_selection import StratifiedKFold , train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc , confusion_matrix
import numpy as np
np.random.seed(42)
from sklearn.metrics import roc_curve, auc

### Build the data set from file

In [ ]:
credit_cards=pd.read_csv('../input/creditcard.csv')
columns=credit_cards.columns
# The labels are in the last column ('Class').

In [ ]:
import numpy as np
credit_cards['Amount'] = np.log(credit_cards['Amount'] + 1)
credit_cards['Time'] = np.log(credit_cards['Time'] + 1)

In [ ]:
features_train = credit_cards

## Isolation forest (anomaly detection)
* https://www.kaggle.io/svf/1100683/56c8356ed1b0a6efccea8371bc791ba7/__results__.html#Tree-based-techniques
) 
* __contamination__ = % of anomalies expected

In [ ]:
model = IsolationForest(random_state=42,  max_samples=0.96, bootstrap=True, n_estimators=90,contamination= 0.002)
model.fit(features_train.drop('Class', axis=1))

In [ ]:
print(model.decision_function(credit_cards[credit_cards['Class'] == 0].values).mean())

In [ ]:
print(model.decision_function(credit_cards[credit_cards['Class'] == 1].values).mean())

* View anomaly score vs each class:
    * (Commented out)

In [ ]:
# print(model.decision_function(credit_cards[credit_cards['Class'] == 0].drop('Class', axis=1).values).mean())
# print(model.decision_function(credit_cards[credit_cards['Class'] == 1].drop('Class', axis=1).values).mean())

In [ ]:
### decision function fails on kaggle for some reason
# credit_cards["isolation_score"] = model.decision_function(credit_cards.drop('Class', axis=1).values)
credit_cards["isolation_score"] = model.predict(credit_cards.drop('Class', axis=1).values)

### Build train and test sets (20% of data reserved to test set)
* Run with or without dropping the __isolation_score__ and compare results! 
* Be sure to keep seed fixed for consistency
* downsampling method may affect results

In [ ]:
features=credit_cards.drop(['Class'],axis=1)
labels=credit_cards['Class']

features_train, features_test, labels_train, labels_test = train_test_split(features, 
                                                                            labels, 
                                                                            test_size=0.2, 
                                                                            random_state=0)

In [ ]:
labels_train.shape

### Perform training of the random forest using the dataset

#### Model without isolation forest feature:

In [ ]:
clf=RandomForestClassifier(random_state=0,class_weight="balanced",n_estimators=300,criterion='entropy')
clf.fit(features_train.drop("isolation_score",axis=1),labels_train)

In [ ]:
# perform predictions on test set
actual=labels_test
predictions=clf.predict(features_test)

In [ ]:
confusion_matrix(actual,predictions)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(actual, predictions)
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

# Rerun with Isolation forest/anomaly metafeature:





In [ ]:
clf=RandomForestClassifier(random_state=0,class_weight="balanced",n_estimators=300,criterion='entropy')
clf.fit(features_train,labels_train)

actual=labels_test
predictions=clf.predict(features_test)

In [ ]:
confusion_matrix(actual,predictions)

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(actual, predictions)
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

### We improved our results by using  the  isolation forest score as a new metafeature! 

Final results locally:

* With isolation forest feat: 91.08  AUC
* Without: 0.90587 AUC